---

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Oracle_logo.svg/2560px-Oracle_logo.svg.png" width="200" align = "left"></p>

# **<h1 align ="middle"><b> Example Lidar Processing</b></h1>**


---

In [ ]:
# !odsc conda init -b conda_environment_yolov5  -n frqap2zhtzbe -a resource_principal 
# !odsc conda publish -s tensorflow28_p38_gpu_v1 --force

---

# **1. Small example - Convert Las file and Create visualisation**

In [ ]:
import pylas 

las = pylas.read('./input_files/in_las_1.las')    # use './in2017_04101720_12.las'
las = pylas.convert(las)    
las.write('out.las')

In [ ]:
import numpy as np
import laspy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# reading las file and copy points
input_las = laspy.read('out.las')
point_records = input_las.points.copy()

In [ ]:
x_mask_g = np.array(point_records.X)
y_mask_g = np.array(point_records.Y)
z_mask_g = np.array(point_records.Z)

x = x_mask_g[:1000]
y = y_mask_g[:1000]
z = z_mask_g[:1000]

fig_surf_g = plt.figure()
ax = fig_surf_g.add_subplot(111, projection = '3d')
ax.plot_trisurf(x, y, z)
plt.title("Ground Surface")

plt.savefig("./save.png")

---

# **2. One script**

In [ ]:
!rm -r ./input_files
!rm -r ./output_files

In [ ]:
%%writefile main.py
import glob
import pylas 
import numpy as np
import laspy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
import ocifs


#create local folder
path_input_locally = "./input_files/" 

try:       
    if not os.path.exists(path_input_locally):         
        os.makedirs(path_input_locally)    

except OSError: 
    print ('Error: Creating directory files ocally')

    
path_output_locally = "./output_files/" 

try:       
    if not os.path.exists(path_output_locally):         
        os.makedirs(path_output_locally)    

except OSError: 
    print ('Error: Creating directory files ocally')


#get files from bucket and store locally

def get_files_from_input_bucket(full_input_bucket):
        
    #get the files from the bucket and store locally
    fs = ocifs.OCIFileSystem()
    
    #invalidate cache
    fs.invalidate_cache(full_input_bucket)
    
    #copy files
    all_files_in_bucket = fs.ls(full_input_bucket) #all files, including files that are not selected to run in experiment
    print(all_files_in_bucket)
    
    #fetch files
    fs.get(full_input_bucket, path_input_locally, recursive=True, refresh=True)  #store files in the bucket in "./" in Job Block storage
    
    return all_files_in_bucket

#call function
all_files_in_bucket = get_files_from_input_bucket("oci://West_BP@frqap2zhtzbe/lidar_data/")

#get file names only
file_list = glob.glob(path_input_locally + "*.las")

loopx = 1

#### read files and convert file file
for file in file_list:
    
    file_name = file
    file_output_name = file
    
    print()
    print("----------------------")
    print("file name " + file_name)
    print("file output name, after conversion" + file_output_name)
    
    print("Convert " + file_name)
    
    #read and convert las to laz
    las = pylas.read(file_name)    
    las = pylas.convert(las)    
    las.write(file_output_name)
    
    # reading las file and copy points
    print("Read " + file_output_name)
    input_las = laspy.read(file_output_name)
    point_records = input_las.points.copy() 
    

    x_mask_g = np.array(point_records.X)
    y_mask_g = np.array(point_records.Y)
    z_mask_g = np.array(point_records.Z)

    x = x_mask_g[:90000]
    y = y_mask_g[:90000]
    z = z_mask_g[:90000]   

    fig_surf_g = plt.figure()
    ax = fig_surf_g.add_subplot(111, projection = '3d')
    ax.plot_trisurf(x, y, z)
    out_name = "./output_files/" + "file_" + str(loopx) + ".png"
    print("save image " + out_name)
    plt.savefig(out_name)
    
    loopx += 1
    print("----------------------")

---

# **3. Create OCI Data Science Job**

In [ ]:
from ads.jobs import Job, DataScienceJob, PythonRuntime

job = (
    Job(name="lidar_example_job")
    .with_infrastructure(
        DataScienceJob()
        .with_log_group_id("ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyajxalcuggjaug57r3ugare7olsk44ts2shyv7azqbxf4q")                                                                                                 ###### Add here your Log Group OCID
        .with_shape_name("VM.GPU.A10.1")                                                                                                 
        #.with_shape_config_details(memory_in_gbs=16, ocpus=5)
        .with_block_storage_size(3000)
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda("oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1")                                                                                             
        .with_source("/home/datascience/1_projects/lidar/main.py") 
        .with_output("output_files", "oci://West_BP@frqap2zhtzbe/lidar_data/ouput")
    )
)

job.create()

# **4. Run Job**

In [ ]:
## create buckets, input and output bucket. Upload .las file to input bucket

In [ ]:
job_run = job.run()

In [ ]:
job_run.watch()